In [8]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
tf.device('/device:GPU.0')
a = tf.constant([[1, 2, 3],[4,5,6]])

b = tf.constant([[1, 2], [3,4], [5,6]])
c = tf.matmul(a,b)
print(c)
tf.__version__

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22 28]
 [49 64]], shape=(2, 2), dtype=int32)


'2.10.0'

## Images path
Point to the folder where the images are

In [4]:
images_path = "../../ImagenesPractica/xview_recognition/"

In [3]:
import uuid
import numpy as np

class GenericObject:
    """
    Generic object data.
    """
    def __init__(self):
        self.id = uuid.uuid4()
        self.bb = (-1, -1, -1, -1)
        self.category= -1

class GenericImage:
    """
    Generic image data.
    """
    def __init__(self, filename):
        self.filename = filename
        self.tile = np.array([-1, -1, -1, -1])  # (pt_x, pt_y, pt_x+width, pt_y+height)
        self.objects = list([])

    def add_object(self, obj: GenericObject):
        self.objects.append(obj)

## Define categories
Define the categories of the images (DON'T MODIFY)

In [2]:
categories = {13: 'CARGO_PLANE', 15: 'HELICOPTER', 18: 'SMALL_CAR', 19: 'BUS', 23: 'TRUCK', 41: 'MOTORBOAT', 47: 'FISHING_VESSEL', 60: 'DUMP_TRUCK', 64: 'EXCAVATOR', 73: 'BUILDING', 86: 'STORAGE_TANK', 91: 'SHIPPING_CONTAINER'}

## Transformations for data augmentation
Operations of rotation and flip.  
For each image we will rotate 90 degrees 3 times, then we will flip the original image and rotate it 90 degrees another 3 times more.  
For each image we will get 8 more.

In [22]:
def rotate_left(img):
    rotated_image = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return rotated_image

def rotate_right(img):
    rotated_image = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    return rotated_image

def rotate_180(img):
    rotated_image = cv2.rotate(img, cv2.ROTATE_180)
    return rotated_image

def flip_image(img):
    flipped_image = cv2.flip(img, 1)
    return flipped_image

In [23]:
def save_image(my_image, filename):
    cv2.imwrite(images_path+filename, my_image)

## Code to perform data augmentation for an image
This code will create the 8 different images and also create the json data for each transformation

In [36]:
import cv2

def create_img_json(base_image_id, base_file_name, new_ending):
    new_img_json = {
        'id': base_image_id + new_ending,
        'file_name': base_file_name + new_ending + '.tif',
        'width': 224, 
        'height': 224
    }
    return new_img_json

def create_ann_json(base_image_id, category_id, new_ending):
    new_ann_json = {
        'id': 1,
        'image_id': base_image_id + new_ending,
        'category_id': category_id,
        'bbox': [0, 0, 224, 224], 
        'iscrowd': 0
    }
    return new_ann_json

def create_new_images(json_img, json_ann):
    new_json_imgs = []
    new_json_anns = []
    
    base_image_id = json_img['id']
    base_file_name = json_img['file_name'].replace(".tif","")
    category_id = json_ann['category_id']
    
    # Read original image
    original_image = cv2.imread(images_path+base_file_name+".tif")
    
    # Rotate image left and save image and create new json data
    rotated_left_image = rotate_left(original_image)
    save_image(rotated_left_image, base_file_name+'rli.tif')
    rli_img_json = create_img_json(base_image_id, base_file_name, 'rli')
    rli_ann_json = create_ann_json(base_image_id, category_id, 'rli')
    new_json_imgs.append(rli_img_json)
    new_json_anns.append(rli_ann_json)
    
    # Rotate image right and save image and create new json data
    rotated_right_image = rotate_right(original_image)
    save_image(rotated_right_image, base_file_name+'rri.tif')
    rri_img_json = create_img_json(base_image_id, base_file_name, 'rri')
    rri_ann_json = create_ann_json(base_image_id, category_id, 'rri')
    new_json_imgs.append(rri_img_json)
    new_json_anns.append(rri_ann_json)
    
    # Rotate image 180 degrees and save image and create new json data
    rotated_180_image = rotate_180(original_image)
    save_image(rotated_180_image, base_file_name+'rii.tif')
    rii_img_json = create_img_json(base_image_id, base_file_name, 'rii')
    rii_ann_json = create_ann_json(base_image_id, category_id, 'rii')
    new_json_imgs.append(rii_img_json)
    new_json_anns.append(rii_ann_json)
    
    # Flip and save image and create new json data
    flipped_image = flip_image(original_image)
    save_image(flipped_image, base_file_name+'fi.tif')
    fi_img_json = create_img_json(base_image_id, base_file_name, 'fi')
    fi_ann_json = create_ann_json(base_image_id, category_id, 'fi')
    new_json_imgs.append(fi_img_json)
    new_json_anns.append(fi_ann_json)
    
    # Rotate fliped left and save image and create new json data
    flipped_rotated_left_image = rotate_left(flipped_image)
    save_image(flipped_rotated_left_image, base_file_name+'fli.tif')
    fli_img_json = create_img_json(base_image_id, base_file_name, 'fli')
    fli_ann_json = create_ann_json(base_image_id, category_id, 'fli')
    new_json_imgs.append(fli_img_json)
    new_json_anns.append(fli_ann_json)
    
    # Rotate flipped right and save image and create new json data
    flipped_rotated_right_image = rotate_right(flipped_image)
    save_image(flipped_rotated_right_image, base_file_name+'fri.tif')
    fri_img_json = create_img_json(base_image_id, base_file_name, 'fri')
    fri_ann_json = create_ann_json(base_image_id, category_id, 'fri')
    new_json_imgs.append(fri_img_json)
    new_json_anns.append(fri_ann_json)
    
    # Rotate flipped 180 degrees and save image and create new json data
    flipped_rotated_180_image = rotate_180(flipped_image)
    save_image(flipped_rotated_180_image, base_file_name+'fii.tif')
    fii_img_json = create_img_json(base_image_id, base_file_name, 'fii')
    fii_ann_json = create_ann_json(base_image_id, category_id, 'fii')
    new_json_imgs.append(fii_img_json)
    new_json_anns.append(fii_ann_json)
    
    return new_json_imgs, new_json_anns

## Load our data base
Be sure to pick the correct json. DO THIS WITH **train.json**

In [5]:
import json
# Load database
json_file = images_path + 'xview_ann_train.json'
with open(json_file) as ifs:
    json_data = json.load(ifs)
ifs.close()

## Let's take a look at our data
We have a lot of images for SMALL_CAR, BUS, TRUCK, and BUILDING. We need more of the rest of categories.  
The output of this cell shows the frequency of each class **before** data augmentation.

In [6]:
import json
import numpy as np
counts = dict.fromkeys(categories.values(), 0)
for json_img, json_ann in zip(json_data['images'], json_data['annotations']):
    obj = GenericObject()
    obj.category = list(categories.values())[json_ann['category_id']-1]
    counts[obj.category] += 1

print(counts)

{'CARGO_PLANE': 5024, 'HELICOPTER': 392, 'SMALL_CAR': 195133, 'BUS': 6549, 'TRUCK': 10640, 'MOTORBOAT': 9848, 'FISHING_VESSEL': 5888, 'DUMP_TRUCK': 9904, 'EXCAVATOR': 5648, 'BUILDING': 283491, 'STORAGE_TANK': 11696, 'SHIPPING_CONTAINER': 12176}


## Data augmentation
We will perform data augmentation now. Don't run this more than once.

In [8]:
import json
import numpy as np
counts = dict.fromkeys(categories.values(), 0)
anns = []
new_images = []
new_annotations = []
for json_img, json_ann in zip(json_data['images'], json_data['annotations']):
    obj = GenericObject()
    obj.category = list(categories.values())[json_ann['category_id']-1]
    
    # Data augmentation for categories that have low images
    if obj.category not in ['SMALL_CAR', 'BUS', 'TRUCK', 'BUILDING']:
        print(json_img['file_name'])
        # augmented_images, augmented_annotations = create_new_images(json_img, json_ann)
        # new_images += augmented_images
        # new_annotations += augmented_annotations
        print("STOP")
    counts[obj.category] += 1
    
# json_data['images'] += new_images
# json_data['annotations'] += new_annotations

# with open(json_file, "w") as file:
    # json.dump(json_data, file)

train/1072/597435c6-14e1-4f00-95da-c6d59f302e8f.tif
STOP
train/1072/1d471153-94ab-45fb-95fb-33e82e32e064.tif
STOP
train/2293/ba496026-8864-4e9a-a518-3e5cf9d8ae9b.tif
STOP
train/2293/911c2ab5-fd81-462f-9095-72b26e73b413.tif
STOP
train/2293/c15f3a8d-ff29-4edc-b303-b111009159ab.tif
STOP
train/2293/76d2850a-5617-4f4b-b44f-2e070d91b213.tif
STOP
train/2293/e011e83f-4a67-40b9-b664-eba2ba346483.tif
STOP
train/2293/89ccb8b6-71a8-4a38-93dc-30850d8a21aa.tif
STOP
train/2293/2ac096f5-6329-48d9-b7bd-8d574bc4a8f8.tif
STOP
train/2293/646b840f-8ef0-4230-905e-58b0a1d49f9d.tif
STOP
train/2293/c99fbc8b-afe4-455b-ade5-be6fb3225d54.tif
STOP
train/2293/8c809a67-23f3-49e6-8538-963ebc641762.tif
STOP
train/2293/d0b574cf-3417-4464-b02e-0ead1f348da8.tif
STOP
train/2293/fc185977-9cc1-4ede-b083-a892d10729cd.tif
STOP
train/2293/c16249fe-35b4-4b08-b589-83515a4e1ae3.tif
STOP
train/2293/74e4cc3d-a947-4da8-aff7-33eefe3f6401.tif
STOP
train/2293/80954d5f-ace8-40ae-ac5b-6111a6f7cd80.tif
STOP
train/2293/ab66b4ff-381e-4be5-9

train/2355/25eaa231-54cf-435d-9ad7-d0ae3822e14d.tif
STOP
train/2355/5b680a32-8b15-4ef6-95b8-4034eeb5d4f2.tif
STOP
train/2355/5e9a1cf9-87db-4071-8501-31a24747b85c.tif
STOP
train/2355/968f7cbf-e535-414e-b1eb-9092154872a1.tif
STOP
train/2355/1fa1e490-7321-4ce5-80d4-8eb60743fbb4.tif
STOP
train/2355/e58e49bc-e53c-4526-8f28-fcad54fec0e4.tif
STOP
train/2355/d44cda58-0082-42cc-a375-7f6e6b5aa7c7.tif
STOP
train/2355/02f98d8c-107a-47c7-b37e-d6f4d8b894bb.tif
STOP
train/2355/264a1ccf-b191-4309-8369-0482840daa14.tif
STOP
train/1921/4a397336-fdba-4213-95a7-cc73b0bd54d5.tif
STOP
train/1921/c1958f50-ca14-46a0-b68d-d71cd4ed0832.tif
STOP
train/1921/397d2f78-a2ef-4f02-ac85-5ee390dd325c.tif
STOP
train/1921/d813c731-4b49-4d56-9d22-faa5b7f10ecd.tif
STOP
train/2513/6f7b2326-9d9f-4140-b12f-0aa383500a95.tif
STOP
train/2513/bd3c3d1f-8a7d-4e7f-b2c6-dcf51444adda.tif
STOP
train/2513/a47344f1-089c-4383-a8bf-e2c41fdfbbff.tif
STOP
train/2513/ded6ea3a-1338-404c-af49-6170d7364a1c.tif
STOP
train/2513/3b851444-97ea-49f0-b

train/1450/0b8be8a9-e3d1-41ea-a8ae-9665e7f9b8a8.tif
STOP
train/1450/1a13100a-fd8a-4d5d-8d1d-fcce747c43e0.tif
STOP
train/1450/9c356c86-e7cb-4c30-8d0e-3b95b43a9169.tif
STOP
train/1450/d8ef6da7-c12d-4963-8cc1-4ceda982baa8.tif
STOP
train/1450/956f33d9-53e2-4cbe-b5d3-3777788b747e.tif
STOP
train/1450/30d00dc5-07ee-454b-bff9-e94854b01d34.tif
STOP
train/1450/367dca32-d628-400a-8fad-6288916843d1.tif
STOP
train/1450/9583fa1e-cf76-4e9e-9c3c-b22d10d4fd6c.tif
STOP
train/1450/2906bba2-563c-46b9-bbbc-3f2efecb2575.tif
STOP
train/1450/8d1da8b9-44b4-46ae-8881-6249e74bfc36.tif
STOP
train/1450/0987b5ce-b8c5-4e18-9cf6-281bfbf613ce.tif
STOP
train/1450/87f1f0de-9486-4c9b-a1d7-bfb362d71ba4.tif
STOP
train/1450/9a7b837e-5889-4dec-82a4-c8057b737d20.tif
STOP
train/1450/18453357-1bd9-452a-a7a3-7c4283143cfd.tif
STOP
train/1450/3aa6347c-b386-46dc-a1f9-2742b8c5b871.tif
STOP
train/1450/bf3c4112-4dfc-48bc-a4aa-bdfdb106f594.tif
STOP
train/1450/cafba9df-d479-4578-9f09-e333f6ef2714.tif
STOP
train/1450/283c978d-0761-4c31-a

train/107/5be5b35d-bf41-4e23-ab58-ae5f33b5cb86.tif
STOP
train/107/22b83e3a-f847-4d99-bd3a-0f9204ff61fb.tif
STOP
train/107/5efb2cba-deaa-4af5-828a-c52a19c22a78.tif
STOP
train/107/5faa7689-3b76-4c74-b1e8-f4b257b1e06b.tif
STOP
train/107/51448902-450e-4041-b099-1c4c4f600fd5.tif
STOP
train/107/f0182a9e-186e-405d-a8cb-9248e0df3bff.tif
STOP
train/107/49b4048f-c3be-4aa6-ba13-1e075a09d803.tif
STOP
train/107/bd08bf5f-0d8f-43da-ab34-aab232ddf482.tif
STOP
train/107/6fb498ca-e5e8-4eae-83d3-2200a57dd35d.tif
STOP
train/107/da9e0bf3-d59e-4356-bd87-b1840afc4fc1.tif
STOP
train/107/fa238bef-b975-4a3b-bd97-44ae1bd5f96d.tif
STOP
train/107/9ff46b7b-1543-4a1c-8799-f472264d4656.tif
STOP
train/107/69fab50a-a720-4e52-8717-d1fffc6138a7.tif
STOP
train/107/cc9c821b-e3bb-4794-b8c4-924edc6335d5.tif
STOP
train/107/3db416af-95eb-4eef-abf9-0f12e8069d97.tif
STOP
train/107/2b711943-a9f7-467d-ade7-f14ac45b6908.tif
STOP
train/107/dc54f6e0-0eb5-4885-8006-5370a7a6d74d.tif
STOP
train/107/18d10b2c-b9ed-45b8-9bce-2a499076c71b.t

train/2026/66c6d7e0-668f-46c5-8f71-e46b9534850c.tif
STOP
train/2026/4a682dd6-7cbb-4715-a4db-c7a0a9d41cab.tif
STOP
train/1920/f8674dc4-b1d5-4062-b8bc-748ceb25d7ba.tif
STOP
train/1920/3204b763-4b57-449e-9a41-2afa7ff5e73d.tif
STOP
train/1920/d53b0045-6952-4fb6-b7a6-3444bc53439d.tif
STOP
train/1920/1fd5406a-ac38-4274-89ec-8a045ed91358.tif
STOP
train/1920/76d375fb-d8e8-4244-90fa-020db767b41a.tif
STOP
train/1920/e2fa1448-253a-47df-80e3-9fd2b39c1c6c.tif
STOP
train/1920/b020a1e0-2832-4a2e-af56-780cfa2c14ca.tif
STOP
train/1920/b9a142ea-1051-44cd-aa72-f75d54c49df9.tif
STOP
train/1920/1147c6ec-281f-45eb-8e6e-f70e57cf83c5.tif
STOP
train/1920/8e600bb0-14d5-45f5-80e9-508a952b0dbd.tif
STOP
train/1920/b3d32103-6c56-4150-897a-0c43d315b18b.tif
STOP
train/1920/022a3857-88fa-4f76-b867-feb905e303cf.tif
STOP
train/1920/aff5c046-b90c-4cb3-88db-52f04170b6ee.tif
STOP
train/1920/5759f5bd-4964-4df8-a4f5-f11135e60712.tif
STOP
train/1920/a1419be1-6ccb-4133-a132-ae44751fa246.tif
STOP
train/1920/8a8c0e73-5027-4c2c-a

train/1217/c1a4c65a-c605-44f8-bf61-bf35f259aa24.tif
STOP
train/1217/434d5051-19db-4efe-a2fa-44bd62421732.tif
STOP
train/1217/b202c720-a1fb-4b0b-977a-ca6fa1e79461.tif
STOP
train/1217/9776804d-98b3-4e3b-bb64-acbdc84b9d24.tif
STOP
train/1217/ed04dbc8-951e-4bba-8f5c-0e6bfc829935.tif
STOP
train/1217/0d2b0be8-33ec-4773-b513-f8344cf27679.tif
STOP
train/1217/a0383442-a982-4793-b732-36c4f4e563cd.tif
STOP
train/1217/334177bb-2358-4535-af5c-6add7fc7db2c.tif
STOP
train/1217/7365f968-77e2-4f6a-abbf-df08f8bcb998.tif
STOP
train/1217/6f70f162-dec6-4a66-bfe8-9e24b8369d18.tif
STOP
train/1217/4e2d752c-085a-4bd6-b340-6fac4f655a7a.tif
STOP
train/1217/06ed65e5-fee2-468a-ba14-93df8f4be49f.tif
STOP
train/1217/cbe64531-95bd-440f-af92-7a54baf0c0b6.tif
STOP
train/1217/e1574b02-b911-40e1-b261-596a67481596.tif
STOP
train/1217/419ba84a-c8c0-4060-a05b-2df632c51ede.tif
STOP
train/1217/16a219c1-207c-4a5b-830e-9085d3551059.tif
STOP
train/1217/38eca228-d33c-41a3-a4d6-2a7474bf19bd.tif
STOP
train/1217/0d477503-08ad-4f48-9

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Results
In the below cell the new frequencies are shown after the data augmentation

In [42]:
import json
import numpy as np
    
counts = dict.fromkeys(categories.values(), 0)
for json_img, json_ann in zip(json_data['images'], json_data['annotations']):
    obj.category = list(categories.values())[json_ann['category_id']-1]
    counts[obj.category] += 1

print(counts)

{'CARGO_PLANE': 5024, 'HELICOPTER': 392, 'SMALL_CAR': 195133, 'BUS': 6549, 'TRUCK': 10640, 'MOTORBOAT': 9848, 'FISHING_VESSEL': 5888, 'DUMP_TRUCK': 9904, 'EXCAVATOR': 5648, 'BUILDING': 283491, 'STORAGE_TANK': 11696, 'SHIPPING_CONTAINER': 12176}
